In [2]:
import pandas as pd

customer      = pd.read_csv("./dump_files/customer_join_with_membership_period.csv")
uselog_months = pd.read_csv("./dump_files/use_log_months.csv")

print("* customer :")
display(customer)

uselog_months = uselog_months.set_index(["customer_id", "연월"])
print("* uselog_months :")
display(uselog_months)

* customer :


,customer_id,name,class,class_name,price,gender,start_date,end_date,membership_period,is_deleted,campaign_id,campaign_name,mean,median,max,min,routine_flg
0,OA832399,XXXX,C01,0_종일,10500,F,2015-05-01,2019-04-30,48.7,0,CA1,2_일반,4.833333,5.0,8,2,1
1,PL270116,XXXXX,C01,0_종일,10500,M,2015-05-01,2019-04-30,48.7,0,CA1,2_일반,5.083333,5.0,7,3,1
2,OA974876,XXXXX,C01,0_종일,10500,M,2015-05-01,2019-04-30,48.7,0,CA1,2_일반,4.583333,5.0,6,3,1
3,HD024127,XXXXX,C01,0_종일,10500,F,2015-05-01,2019-04-30,48.7,0,CA1,2_일반,4.833333,4.5,7,2,1
4,HD661448,XXXXX,C03,2_야간,6000,F,2015-05-01,2019-04-30,48.7,0,CA1,2_일반,3.916667,4.0,6,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,0_종일,10500,M,2019-03-14,2019-04-30,1.6,0,CA1,2_일반,8.000000,8.0,8,8,0
4188,HD246549,XXXXX,C01,0_종일,10500,F,2019-03-14,2019-04-30,1.6,0,CA1,2_일반,10.000000,10.0,10,10,0
4189,GD037007,XXXXX,C03,2_야간,6000,M,2019-03-14,2019-04-30,1.6,0,CA1,2_일반,8.000000,8.0,8,8,0
4190,OA953150,XXXXX,C01,0_종일,10500,M,2019-03-14,2019-04-30,1.6,0,CA1,2_일반,11.000000,11.0,11,11,0


* uselog_months :


,,count
customer_id,연월,
AS002855,201804,4
AS009013,201804,2
AS009373,201804,3
AS015315,201804,6
AS015739,201804,7
...,...,...
TS995853,201903,8
TS998593,201903,8
TS999079,201903,3


In [14]:
# 기준 연월과 해당 기준 연월의 1개월 전 이용 횟수를 집계한 데이터 작성
# '연월'값에 '201804'이전 데이터는 존재하지 않기 때문에, 연월값이 '201804'인 레코드를 조회에서 제외함

from pandasql import sqldf

uselog_base_month_cnt_and_bfr_month_cnt = sqldf("""
                                            SELECT Z.*
                                            FROM (
                                                SELECT X.연월,
                                                    X.customer_id,
                                                    X.count AS count_0,
                                                    (SELECT Y1.count 
                                                     FROM   uselog_months Y1 
                                                     WHERE  Y1.customer_id = X.customer_id 
                                                            AND Y1.연월 = STRFTIME('%Y%m', DATETIME(SUBSTR(X.연월, 1, 4) || '-' || SUBSTR(X.연월, 5, 2) || '-' || '01', '-1 months'))
                                                    ) AS count_1        
                                                FROM   uselog_months X  
                                            ) Z
                                            WHERE Z.연월 != '201804'     
                                        """)
print("* uselog_base_month_cnt_and_bfr_month_cnt :")
display(uselog_base_month_cnt_and_bfr_month_cnt)      

# # 기준 연월과 해당 기준 연월의 1개월 전 이용 횟수를 집계한 데이터 작성
# year_months = list(uselog_months["연월"].unique())
# print("year_months :", year_months)
# uselog = pd.DataFrame()

# for i in range(1, len(year_months)): # 1부터 반복을 도는 이유는 'year_months' 내 '201804'이전 데이터는 존재하지 않기 때문
    
#     tmp = uselog_months.loc[uselog_months["연월"] == year_months[i]]
#     tmp.rename(columns = {"count" : "count_0"}, inplace = True)
#     tmp_before = uselog_months.loc[uselog_months["연월"] == year_months[i-1]]
#     del tmp_before["연월"]
#     tmp_before.rename(columns = {"count" : "count_1"}, inplace = True)
#     tmp = pd.merge(tmp, tmp_before, on = "customer_id", how="left")
#     uselog = pd.concat([uselog, tmp], ignore_index=True)

# display(uselog.head())

* uselog_base_month_cnt_and_bfr_month_cnt :


,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0
...,...,...,...,...
33846,201903,TS995853,8,11.0
33847,201903,TS998593,8,7.0
33848,201903,TS999079,3,2.0
33849,201903,TS999231,6,6.0


In [20]:
# exit_customer, exit_uselog 데이터 프레임 생성
# exit_date : 회원 탈퇴 '신청 일자'

exit_customer = sqldf("""
                        SELECT customer_id,
                               name,
                               class,
                               class_name,
                               price,
                               gender,
                               STRFTIME('%Y%m%d', start_date) AS start_date,
                               STRFTIME('%Y%m%d', end_date) AS end_date,
                               STRFTIME('%Y%m%d', DATETIME(DATETIME(end_date, 'start of month'), '-1 days')) AS exit_date,
                               STRFTIME('%Y%m', DATETIME(DATETIME(end_date, 'start of month'), '-1 days')) AS exit_date_YYYY_MM,
                               membership_period,
                               is_deleted,
                               campaign_id,
                               campaign_name,
                               mean,
                               median,
                               max,
                               min,
                               routine_flg
                        FROM   customer
                        WHERE  is_deleted = 1
                     """)
print("* exit_customer :")
display(exit_customer)

exit_uselog = sqldf("""
                     SELECT    X.customer_id,
                               X.is_deleted,
                               X.start_date,
                               X.end_date,
                               X.exit_date,
                               X.exit_date_YYYY_MM,
                               Y.count_0,
                               Y.count_1
                     FROM      exit_customer X
                     LEFT JOIN uselog_base_month_cnt_and_bfr_month_cnt Y
                     ON        (X.exit_date_YYYY_MM = Y.연월
                                AND X.customer_id = Y.customer_id)

                   """)
print("* exit_uselog :")
display(exit_uselog)


* exit_customer :


,customer_id,name,class,class_name,price,gender,start_date,end_date,exit_date,exit_date_YYYY_MM,membership_period,is_deleted,campaign_id,campaign_name,mean,median,max,min,routine_flg
0,TS511179,XXXXXX,C01,0_종일,10500,F,20160501,20180430,20180331,201803,24.3,1,CA1,2_일반,3.0,3.0,3,3,0
1,TS443736,XXXX,C02,1_주간,7500,M,20160501,20180430,20180331,201803,24.3,1,CA1,2_일반,3.0,3.0,3,3,0
2,HD542886,XX,C01,0_종일,10500,M,20160501,20180430,20180331,201803,24.3,1,CA1,2_일반,1.0,1.0,1,1,0
3,HD597545,XXXXX,C03,2_야간,6000,F,20160601,20180531,20180430,201804,24.3,1,CA1,2_일반,3.5,3.5,4,3,1
4,HI749296,XXXXX,C01,0_종일,10500,M,20160601,20180531,20180430,201804,24.3,1,CA1,2_일반,3.0,3.0,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,OA143589,XXXXX,C03,2_야간,6000,F,20190115,20190228,20190131,201901,1.5,1,CA1,2_일반,4.5,4.5,6,3,0
1346,GD367382,XXXX,C01,0_종일,10500,M,20190115,20190331,20190228,201902,2.5,1,CA1,2_일반,7.0,6.0,12,3,0
1347,OA034442,XXXXX,C02,1_주간,7500,M,20190205,20190331,20190228,201902,1.8,1,CA1,2_일반,7.0,7.0,11,3,0
1348,PL508278,XXXXX,C03,2_야간,6000,M,20190209,20190331,20190228,201902,1.7,1,CA1,2_일반,7.0,7.0,10,4,0


* exit_uselog :


,customer_id,is_deleted,start_date,end_date,exit_date,exit_date_YYYY_MM,count_0,count_1
0,TS511179,1,20160501,20180430,20180331,201803,NaN,NaN
1,TS443736,1,20160501,20180430,20180331,201803,NaN,NaN
2,HD542886,1,20160501,20180430,20180331,201803,NaN,NaN
3,HD597545,1,20160601,20180531,20180430,201804,NaN,NaN
4,HI749296,1,20160601,20180531,20180430,201804,NaN,NaN
...,...,...,...,...,...,...,...,...
1345,OA143589,1,20190115,20190228,20190131,201901,6.0,NaN
1346,GD367382,1,20190115,20190331,20190228,201902,6.0,12.0
1347,OA034442,1,20190205,20190331,20190228,201902,11.0,NaN
1348,PL508278,1,20190209,20190331,20190228,201902,10.0,NaN


In [4]:
# 탈퇴한 회원들의 탈퇴 전월 데이터 작성
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer["is_deleted"] == 1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
# exit_customer.head()
for i in range(len(exit_customer)):
    
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
    
#'exit_customer'데이터 셋 내 'exit_date'를 'YYYYMM'포맷의 문자열로 변경한 후, 이것을 '연월'이라는 컬럼으로 생성함
exit_customer["연월"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on = ["customer_id", "연월"], how="left")
exit_uselog.head()

,연월,customer_id,count_0,count_1,gender,start_date,end_date,is_deleted,class,class_name,campaign_id,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# 'exit_customer'데이터 셋 확인
exit_customer.head()

,customer_id,gender,start_date,end_date,is_deleted,class,class_name,campaign_id,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,연월
708,TS511179,F,2016-05-01,2018-04-30,1,C01,0_종일,CA1,2_일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00,201803
729,TS443736,M,2016-05-01,2018-04-30,1,C02,1_주간,CA1,2_일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00,201803
730,HD542886,M,2016-05-01,2018-04-30,1,C01,0_종일,CA1,2_일반,1.0,1.0,1,1,0,2018-04-30,23,2018-03-30 00:00:00,201803
770,HD597545,F,2016-06-01,2018-05-31,1,C03,2_야간,CA1,2_일반,3.5,3.5,4,3,1,2018-05-31,23,2018-04-30 00:00:00,201804
785,HI749296,M,2016-06-01,2018-05-31,1,C01,0_종일,CA1,2_일반,3.0,3.0,3,3,0,2018-05-31,23,2018-04-30 00:00:00,201804


In [6]:
# 'exit_uselog'데이터 셋 확인
exit_uselog.head()

,연월,customer_id,count_0,count_1,gender,start_date,end_date,is_deleted,class,class_name,campaign_id,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# 'uselog'와 'exit_customer'를 left join하여 'exit_uselog'를 만들었기 때문에, 'exit_uselog'내 결측치를 제거해야지만 해당 데이터 셋에 탈퇴신청한 회원의 데이터만 남게 됨 
exit_uselog = exit_uselog.dropna(subset = ["exit_date"])
print(len(exit_uselog))
exit_uselog.head()

1104


,연월,customer_id,count_0,count_1,gender,start_date,end_date,is_deleted,class,class_name,campaign_id,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,M,2018-03-01,2018-06-30,1.0,C01,0_종일,CA1,2_일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,M,2017-11-01,2018-06-30,1.0,C01,0_종일,CA1,2_일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,M,2017-06-01,2018-06-30,1.0,C01,0_종일,CA1,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,M,2018-02-01,2018-06-30,1.0,C01,0_종일,CA1,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,F,2017-09-01,2018-06-30,1.0,C03,2_야간,CA1,2_일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


In [ ]:
conti_customer = customer.loc[customer["is_deleted"] == 0]
conti_uselog = pd.merge(uselog, conti_customer, on = "customer_id", how="left")
